In [ ]:
import pandas as pd
import numpy as np

data_path = '../../data/btc_usd_h4_features.csv'
df = pd.read_csv(data_path, index_col='timestamp', parse_dates=True)

print("Data loaded successfully.")
display(df.head())

In [ ]:
def define_target(df, atr_multiplier_tp=3.0, atr_multiplier_sl=1.5, lookahead_candles=30):
    df['target'] = 0
    
    # Find the correct ATR column name automatically
    atr_column_name = next((col for col in df.columns if 'ATRr_' in col), None)
    if not atr_column_name:
        raise ValueError("ATR column not found in DataFrame. Please ensure it's calculated in Notebook 1.")
        
    for i in range(len(df) - lookahead_candles):
        entry_price = df['close'].iloc[i]
        atr_value = df[atr_column_name].iloc[i]

        future_window_high = df['high'].iloc[i+1 : i+1+lookahead_candles]
        future_window_low = df['low'].iloc[i+1 : i+1+lookahead_candles]
        
        take_profit_buy = entry_price + (atr_value * atr_multiplier_tp)
        stop_loss_buy = entry_price - (atr_value * atr_multiplier_sl)
        
        take_profit_sell = entry_price - (atr_value * atr_multiplier_tp)
        stop_loss_sell = entry_price + (atr_value * atr_multiplier_sl)
        
        buy_profit_hit_time = future_window_high[future_window_high >= take_profit_buy].first_valid_index()
        buy_loss_hit_time = future_window_low[future_window_low <= stop_loss_buy].first_valid_index()
        
        if buy_profit_hit_time is not None and (buy_loss_hit_time is None or buy_profit_hit_time < buy_loss_hit_time):
            df.loc[df.index[i], 'target'] = 1
            continue

        sell_profit_hit_time = future_window_low[future_window_low <= take_profit_sell].first_valid_index()
        sell_loss_hit_time = future_window_high[future_window_high >= stop_loss_sell].first_valid_index()
        
        if sell_profit_hit_time is not None and (sell_loss_hit_time is None or sell_profit_hit_time < sell_loss_hit_time):
            df.loc[df.index[i], 'target'] = -1
            continue
            
    return df

print("New dynamic define_target function created and ready.")

In [ ]:
labeled_df = define_target(df)

print("Target labeling complete.")

target_distribution = labeled_df['target'].value_counts()
print("\nTarget Distribution:")
print(target_distribution)

target_distribution.plot(kind='bar', title='Distribution of Target Labels (0=Hold, 1=Buy, -1=Sell)')

In [ ]:
output_path = '../../data/btc_usd_h4_labeled.csv'
labeled_df.to_csv(output_path)

print(f"Labeled data saved successfully to: {output_path}")